#                   EVALUATION OF LSTM                                                                                    ' TENSORBOARD - HYPERPARAMETERS ' 

# PHASE I: DATA PRE - PROCESSING

In [21]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [22]:
# importing libraries
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import time

In [23]:
# Loading the whole dataset and setting Date column as an index

dataset = pd.read_csv(r'C:\Users\Pamela\Desktop\PROJECT\DATASET NASDAQ 100\nasdaq100_joined_closes.csv', index_col=0)

In [24]:
# In this case study Amazon 'AMZN' company is selected: 

dataset_1 = dataset.iloc[0:, 7:8]
dataset_1.head()
dataset_1.shape

,AMZN
Date,
04/01/2010,133.899994
05/01/2010,134.690002
06/01/2010,132.250000
07/01/2010,130.000000
08/01/2010,133.520004


(2335, 1)

In [25]:
dataset_f = dataset_1.iloc[1258:]

In [26]:
#Dividing the dataset between training and testing datasets and selecting just the 'Adj Close' column.
training_set = dataset_f.iloc[0:1063].values
testing_set = dataset_f.iloc [1063: ].values


In [27]:
# Exploring and exporting 'training_set'
type (training_set)
training_set.shape


numpy.ndarray

(1063, 1)

In [28]:
# Exploring and exporting 'testing_set'
type (testing_set)
testing_set.shape


numpy.ndarray

(14, 1)

In [29]:
# Feature Scaling betweaen 0 and 1
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0,1))
training_set_scaled = sc.fit_transform(training_set)

In [30]:
# To check the data in 'training_set_scaled'
training_set_scaled.shape
type(training_set_scaled[0:60,0])


(1063, 1)

numpy.ndarray

In [31]:
# Creating a data structure with 60 timesteps and 1 output
X_train = []
y_train = []
for i in range (60,len(training_set)):
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])

# converting the lists 'X_train' and 'y_train' in numpy array
X_train, y_train = np.array(X_train), np.array(y_train)

In [32]:
# To check the data in 'X_train'
type (X_train)
X_train.shape


numpy.ndarray

(1003, 60)

In [33]:
# To check the data in 'y_train'
type (y_train)
y_train.shape


numpy.ndarray

(1003,)

In [34]:
# Reshaping --> in Dimention 3D (Batch size, timesteps, input_dim)
X_train_R = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_train_R.shape

(1003, 60, 1)

## PHASE 4: TRAINING MODEL 
## Creating the RNN networK AND APPLYING HYPERPARAMETERS -TENSORBOARD

In [39]:
#Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Activation
#import time

# To implement TensorBoard
from keras.callbacks import TensorBoard

In [40]:
first_neurons = [50, 100, 150]
hidden_neurons = [50, 100, 150]
batches_size = [64, 128]
epochs = [18, 25, 35]


In [41]:
for first_neuron in first_neurons:
    for hidden_neuron in hidden_neurons:
        for batch_size1 in batches_size:
            for epoch in epochs:
                NAME = "{}-epoch-{}-batch-{}-hidden_neuron-{}-first_neuron-{}".format(epoch, batch_size1, hidden_neuron, 
                        first_neuron, int(time.time()))
                print(NAME)

18-epoch-64-batch-50-hidden_neuron-50-first_neuron-1560274862
25-epoch-64-batch-50-hidden_neuron-50-first_neuron-1560274862
35-epoch-64-batch-50-hidden_neuron-50-first_neuron-1560274862
18-epoch-128-batch-50-hidden_neuron-50-first_neuron-1560274862
25-epoch-128-batch-50-hidden_neuron-50-first_neuron-1560274862
35-epoch-128-batch-50-hidden_neuron-50-first_neuron-1560274862
18-epoch-64-batch-100-hidden_neuron-50-first_neuron-1560274862
25-epoch-64-batch-100-hidden_neuron-50-first_neuron-1560274862
35-epoch-64-batch-100-hidden_neuron-50-first_neuron-1560274862
18-epoch-128-batch-100-hidden_neuron-50-first_neuron-1560274862
25-epoch-128-batch-100-hidden_neuron-50-first_neuron-1560274862
35-epoch-128-batch-100-hidden_neuron-50-first_neuron-1560274862
18-epoch-64-batch-150-hidden_neuron-50-first_neuron-1560274862
25-epoch-64-batch-150-hidden_neuron-50-first_neuron-1560274862
35-epoch-64-batch-150-hidden_neuron-50-first_neuron-1560274862
18-epoch-128-batch-150-hidden_neuron-50-first_neuron-15

In [42]:
for first_neuron in first_neurons:
    for hidden_neuron in hidden_neurons:
        for batch_size1 in batches_size:
            for epoch in epochs:
                NAME = "{}-epoch-{}-batch-{}-hidden_neuron-{}-first_neuron-{}".format(epoch, batch_size1, hidden_neuron, first_neuron, int(time.time()))
                print(NAME)

                model_regressor = Sequential()
                    
                #Adding the initial layer with 'first_neuron' as a variable parameter and dropout regularisation to control overfitting
                model_regressor.add(LSTM(units = first_neuron, return_sequences=True, input_shape = (X_train_R.shape[1], 1)))
                model_regressor.add(Dropout(0.2))
               
                # Hidden Layers
                #First hidden layer LSTM layer and Dropout regularisation to control overfitting
                model_regressor.add(LSTM(units = hidden_neuron, return_sequences=False))
                model_regressor.add(Dropout(0.2))
                                        
                 
                ## final layer
                model_regressor.add(Dense(output_dim=1))
                model_regressor.add(Activation('linear'))
    
                # Compiling the RNN
                model_regressor.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics= ['mae', 'acc']) 

                # Tensorboard
               
                #tensorboard = TensorBoard(log_dir="logs/{}")
                tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))
                    
                model_regressor.fit(X_train_R, y_train, epochs = epoch, batch_size = batch_size1, callbacks=[tensorboard])
                



18-epoch-64-batch-50-hidden_neuron-50-first_neuron-1560274872


C:\Users\Pamela\Anaconda3\envs\hgp\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=1)`


Epoch 1/18
1003/1003 [==============================] - 8s 8ms/step - loss: 0.0814 - mean_absolute_error: 0.2110 - acc: 9.9701e-04
Epoch 2/18
1003/1003 [==============================] - 5s 5ms/step - loss: 0.0124 - mean_absolute_error: 0.0823 - acc: 9.9701e-04
Epoch 3/18
1003/1003 [==============================] - 5s 5ms/step - loss: 0.0055 - mean_absolute_error: 0.0590 - acc: 9.9701e-04
Epoch 4/18
1003/1003 [==============================] - 5s 5ms/step - loss: 0.0032 - mean_absolute_error: 0.0400 - acc: 9.9701e-04
Epoch 5/18
1003/1003 [==============================] - 6s 6ms/step - loss: 0.0028 - mean_absolute_error: 0.0359 - acc: 9.9701e-04
Epoch 6/18
1003/1003 [==============================] - 5s 5ms/step - loss: 0.0029 - mean_absolute_error: 0.0364 - acc: 9.9701e-04
Epoch 7/18
1003/1003 [==============================] - 5s 5ms/step - loss: 0.0029 - mean_absolute_error: 0.0367 - acc: 9.9701e-04
Epoch 8/18
1003/1003 [==============================] - 5s 5ms/step - loss: 0.0030 

25-epoch-64-batch-50-hidden_neuron-50-first_neuron-1560274981
Epoch 1/25
1003/1003 [==============================] - 9s 9ms/step - loss: 0.0767 - mean_absolute_error: 0.2037 - acc: 9.9701e-04
Epoch 2/25
1003/1003 [==============================] - 4s 4ms/step - loss: 0.0109 - mean_absolute_error: 0.0769 - acc: 9.9701e-04
Epoch 3/25
1003/1003 [==============================] - 4s 4ms/step - loss: 0.0052 - mean_absolute_error: 0.0585 - acc: 9.9701e-04
Epoch 4/25
1003/1003 [==============================] - 4s 4ms/step - loss: 0.0033 - mean_absolute_error: 0.0401 - acc: 9.9701e-04
Epoch 5/25
1003/1003 [==============================] - 5s 5ms/step - loss: 0.0031 - mean_absolute_error: 0.0379 - acc: 9.9701e-04
Epoch 6/25
1003/1003 [==============================] - 5s 5ms/step - loss: 0.0027 - mean_absolute_error: 0.0352 - acc: 9.9701e-04
Epoch 7/25
1003/1003 [==============================] - 5s 5ms/step - loss: 0.0029 - mean_absolute_error: 0.0367 - acc: 9.9701e-04
Epoch 8/25
1003/1003 

35-epoch-64-batch-50-hidden_neuron-50-first_neuron-1560275130
Epoch 1/35
1003/1003 [==============================] - 8s 8ms/step - loss: 0.0732 - mean_absolute_error: 0.2026 - acc: 0.0000e+00
Epoch 2/35
1003/1003 [==============================] - 5s 5ms/step - loss: 0.0082 - mean_absolute_error: 0.0675 - acc: 9.9701e-04
Epoch 3/35
1003/1003 [==============================] - 5s 5ms/step - loss: 0.0043 - mean_absolute_error: 0.0494 - acc: 9.9701e-04
Epoch 4/35
1003/1003 [==============================] - 5s 5ms/step - loss: 0.0037 - mean_absolute_error: 0.0414 - acc: 9.9701e-04
Epoch 5/35
1003/1003 [==============================] - 5s 5ms/step - loss: 0.0038 - mean_absolute_error: 0.0408 - acc: 9.9701e-04
Epoch 6/35
1003/1003 [==============================] - 5s 5ms/step - loss: 0.0028 - mean_absolute_error: 0.0373 - acc: 9.9701e-04
Epoch 7/35
1003/1003 [==============================] - 4s 4ms/step - loss: 0.0030 - mean_absolute_error: 0.0384 - acc: 9.9701e-04
Epoch 8/35
1003/1003 

...

25-epoch-64-batch-150-hidden_neuron-150-first_neuron-1560296985
Epoch 1/25
1003/1003 [==============================] - 110s 109ms/step - loss: 0.0422 - mean_absolute_error: 0.1365 - acc: 9.9701e-04
Epoch 2/25
1003/1003 [==============================] - 12s 12ms/step - loss: 0.0037 - mean_absolute_error: 0.0469 - acc: 9.9701e-04
Epoch 3/25
1003/1003 [==============================] - 12s 12ms/step - loss: 0.0018 - mean_absolute_error: 0.0297 - acc: 9.9701e-04
Epoch 4/25
1003/1003 [==============================] - 12s 12ms/step - loss: 0.0014 - mean_absolute_error: 0.0256 - acc: 9.9701e-04
Epoch 5/25
1003/1003 [==============================] - 13s 13ms/step - loss: 0.0014 - mean_absolute_error: 0.0251 - acc: 9.9701e-04
Epoch 6/25
1003/1003 [==============================] - 11s 11ms/step - loss: 0.0015 - mean_absolute_error: 0.0260 - acc: 9.9701e-04
Epoch 7/25
1003/1003 [==============================] - 12s 12ms/step - loss: 0.0015 - mean_absolute_error: 0.0253 - acc: 9.9701e-04
Epo

35-epoch-64-batch-150-hidden_neuron-150-first_neuron-1560297936
Epoch 1/35
1003/1003 [==============================] - 144s 144ms/step - loss: 0.0434 - mean_absolute_error: 0.1357 - acc: 9.9701e-04
Epoch 2/35
1003/1003 [==============================] - 13s 13ms/step - loss: 0.0047 - mean_absolute_error: 0.0536 - acc: 9.9701e-04
Epoch 3/35
1003/1003 [==============================] - 16s 16ms/step - loss: 0.0023 - mean_absolute_error: 0.0340 - acc: 9.9701e-04
Epoch 4/35
1003/1003 [==============================] - 21s 21ms/step - loss: 0.0017 - mean_absolute_error: 0.0277 - acc: 9.9701e-04
Epoch 5/35
1003/1003 [==============================] - 12s 12ms/step - loss: 0.0014 - mean_absolute_error: 0.0256 - acc: 9.9701e-04
Epoch 6/35
1003/1003 [==============================] - 13s 13ms/step - loss: 0.0013 - mean_absolute_error: 0.0246 - acc: 9.9701e-04
Epoch 7/35
1003/1003 [==============================] - 14s 14ms/step - loss: 0.0013 - mean_absolute_error: 0.0248 - acc: 9.9701e-04
Epo

18-epoch-128-batch-150-hidden_neuron-150-first_neuron-1560298990
Epoch 1/18
1003/1003 [==============================] - 100s 100ms/step - loss: 0.0668 - mean_absolute_error: 0.1820 - acc: 9.9701e-04
Epoch 2/18
1003/1003 [==============================] - 9s 9ms/step - loss: 0.0137 - mean_absolute_error: 0.0841 - acc: 9.9701e-04
Epoch 3/18
1003/1003 [==============================] - 10s 10ms/step - loss: 0.0060 - mean_absolute_error: 0.0679 - acc: 9.9701e-04
Epoch 4/18
1003/1003 [==============================] - 10s 10ms/step - loss: 0.0038 - mean_absolute_error: 0.0424 - acc: 9.9701e-04
Epoch 5/18
1003/1003 [==============================] - 10s 10ms/step - loss: 0.0021 - mean_absolute_error: 0.0362 - acc: 9.9701e-04
Epoch 6/18
1003/1003 [==============================] - 10s 10ms/step - loss: 0.0019 - mean_absolute_error: 0.0307 - acc: 9.9701e-04
Epoch 7/18
1003/1003 [==============================] - 10s 10ms/step - loss: 0.0017 - mean_absolute_error: 0.0280 - acc: 9.9701e-04
Epoc

25-epoch-128-batch-150-hidden_neuron-150-first_neuron-1560299654
Epoch 1/25
1003/1003 [==============================] - 263s 262ms/step - loss: 0.0667 - mean_absolute_error: 0.1844 - acc: 9.9701e-04
Epoch 2/25
1003/1003 [==============================] - 10s 10ms/step - loss: 0.0124 - mean_absolute_error: 0.0794 - acc: 9.9701e-04
Epoch 3/25
1003/1003 [==============================] - 12s 12ms/step - loss: 0.0050 - mean_absolute_error: 0.0611 - acc: 9.9701e-04
Epoch 4/25
1003/1003 [==============================] - 11s 11ms/step - loss: 0.0034 - mean_absolute_error: 0.0407 - acc: 9.9701e-04
Epoch 5/25
1003/1003 [==============================] - 11s 11ms/step - loss: 0.0021 - mean_absolute_error: 0.0346 - acc: 9.9701e-04
Epoch 6/25
1003/1003 [==============================] - 11s 11ms/step - loss: 0.0018 - mean_absolute_error: 0.0299 - acc: 9.9701e-04
Epoch 7/25
1003/1003 [==============================] - 10s 10ms/step - loss: 0.0017 - mean_absolute_error: 0.0282 - acc: 9.9701e-04
Ep

35-epoch-128-batch-150-hidden_neuron-150-first_neuron-1560300599
Epoch 1/35
1003/1003 [==============================] - 314s 313ms/step - loss: 0.0664 - mean_absolute_error: 0.1823 - acc: 9.9701e-04
Epoch 2/35
1003/1003 [==============================] - 10s 10ms/step - loss: 0.0116 - mean_absolute_error: 0.0758 - acc: 9.9701e-04
Epoch 3/35
1003/1003 [==============================] - 10s 10ms/step - loss: 0.0047 - mean_absolute_error: 0.0583 - acc: 9.9701e-04
Epoch 4/35
1003/1003 [==============================] - 10s 10ms/step - loss: 0.0034 - mean_absolute_error: 0.0409 - acc: 9.9701e-04
Epoch 5/35
1003/1003 [==============================] - 10s 10ms/step - loss: 0.0022 - mean_absolute_error: 0.0347 - acc: 9.9701e-04
Epoch 6/35
1003/1003 [==============================] - 9s 9ms/step - loss: 0.0017 - mean_absolute_error: 0.0286 - acc: 9.9701e-04
Epoch 7/35
1003/1003 [==============================] - 10s 10ms/step - loss: 0.0016 - mean_absolute_error: 0.0274 - acc: 9.9701e-04
Epoc